In [65]:
import yfinance as yf
import math

In [ ]:
stock = yf.Ticker('TSLA')
stockInfo = stock.info

In [182]:
stock.shares
#stock.info['sharesOutstanding']

,BasicShares
Year,
2019,18471336000
2020,17352119000
2021,16701272000


# EPS

In [67]:
def calc_eps(net_income_to_common, shares_outstanding):
    return net_income_to_common / shares_outstanding

calc_eps(stockInfo['netIncomeToCommon'], stockInfo['sharesOutstanding'])

3.038484681151332

# Volatility
Valid intervals: `[1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo]`

In [ ]:
def calc_volatility(stock, period, interval):
    historic = stock.history(period=period, interval=interval)
    values = [x for x in historic['Close'] if not math.isnan(x)]
    print(historic['Close'], len(historic['Close']))

    mean = sum(values) / len(values)
    deviation = [price - mean for price in values]
    variance = sum([dev * dev for dev in deviation]) / len(values)
    std_dev = math.sqrt(variance)

    #Not completely sure if this is the exact formula that is needed/wanted?
    volatility = std_dev * math.sqrt(len(values))
    
    return volatility
    


calc_volatility(stock, '2mo', '1d')


# Estimate future price

In [187]:
gr_values = {
    'Total Assets': 0.1,
    'Total Current Assets': 0.05,
    'Retained Earnings': 0.05,
    'Gross Profit': 0.2,
    'Net Income': 0.2,
    'Total Cash From Operating Activities': 0.15
}

def calc_growth_rate(stock, eps_an, periods, bs, fc, cf):
    result = 0

    for i, (data, weight) in enumerate(gr_values.items()):
        if i < 3:
            source = bs
        elif i < 5:
            source = fc
        else:
            source = cf
            
        data_gr = 0
        for j, period in enumerate(periods):
            if j == len(periods) - 1:
                break
                    
            present = source[period][data]
            past = source[periods[j + 1]][data]
            data_gr += ((present - past) / past)
            

        result += ((data_gr / 2) * weight)
        
    #Specific for EPS
    eps_gr = 0
    for i, eps in enumerate(eps_an):
        if i == len(periods) - 1:
            break
                    
        past = eps_an[i + 1]
        eps_gr += ((eps - past) / past)
    result += ((eps_gr / 2) * 0.25)

    return result


#returns stockprice in 3 years
def get_future_price(stock, when='3y'):
    if when == '1y':
        financials = stock.quarterly_financials
        bs = stock.quarterly_balancesheet
        cf = stock.quarterly_cashflow
        kwargs = {'period': '1y'}
    elif when =='3y':
        financials = stock.financials
        bs = stock.balancesheet
        cf = stock.cashflow
        kwargs = {'start':'2019-01-01', 'end':'2021-12-31'}
    else:
        print("'when' param should be 1y or 3y")
        return -1

    historic = stock.history(interval='1mo', **kwargs)['Close']
    fc = historic[historic.notnull()]
    periods = financials.keys()[:4]

    eps_an = [round(financials[period]['Net Income Applicable To Common Shares'] / stock.shares['BasicShares'][2021], 2) for period in periods]

    #future p/e ratio
    if when == '1y':
        closes = [fc[ts] for ts in fc.keys() if ts.month % 3 == 0]
    elif when =='3y':
        closes = [fc[ts] for ts in fc.keys() if ts.month == 12]
    closes.reverse()

    tot_pe = [price / eps_an[i] for i, price in enumerate(closes)]
    pe = round(sum(tot_pe) / len(tot_pe), 2)

    #Calculate future eps
    eps = eps_an[0] * ((1 + calc_growth_rate(stock, eps_an, periods, bs, financials, cf)) ** len(periods))

    return pe * eps
    

print(f"Future (3y) price of {stock.info['symbol']}: {get_future_price(stock, '1y')}")

[1.16, 1.5, 2.07, 1.23]
Date
2021-11-01 00:00:00-04:00    164.378906
2021-12-01 00:00:00-05:00    176.838242
2022-01-01 00:00:00-05:00    174.059723
2022-02-01 00:00:00-05:00    164.439545
2022-03-01 00:00:00-05:00    174.111984
2022-04-01 00:00:00-04:00    157.200348
2022-05-01 00:00:00-04:00    148.415482
2022-06-01 00:00:00-04:00    136.530350
2022-07-01 00:00:00-04:00    162.284576
2022-08-01 00:00:00-04:00    157.001923
2022-09-01 00:00:00-04:00    138.199997
2022-10-01 00:00:00-04:00    144.800003
2022-10-27 00:00:00-04:00    144.800003
Name: Close, dtype: float64
[176.83824157714844, 174.1119842529297, 136.5303497314453, 138.1999969482422]
Future (3y) price of AAPL: 207.39551706274975
